# Projeto DIO COVID-19

Super dica... os dados foram retirados no Kaggle.com

## Instalação de biblioteca

In [ ]:
#opcional: instalado por falha na importacao do Plotly, msg modulo inexistente
pip install plotly

In [ ]:
#opcional: pcufflinks concecta o Pandas ao Plotly.
pip install cufflinks

## Importar bibliotecas do Python

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
#bibliotecas de visualização com recrusos em java script
import plotly.express as px  
import plotly.graph_objects as go 

## Importar e visualizar o conjunto de dados

In [ ]:
##Upload do arquivo
#from google.colab import files
#arq = files.upload()
url = "https://github.com/otavioaugust1/DIO_Unimed-BH_CienciaDeDados/blob/main/Desafio/Evolucao_do_COVID19_Brasil/covid_19_data.csv?raw=true"

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#importa campos como data e não como texto
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])

In [ ]:
#chama o conjunto de dados para visualização em javascript
df

In [ ]:
# Renomeando a tabela
df = df.rename(columns={"SNo":"Numero De Serie","ObservationDate":"Data De Observacao","Province/State":"Cidade/Estado","Country/Region":"Pais/Regiao","Last Update":"Ultima Atualização","Confirmed":"Confirmados","Deaths":"Obitos","Recovered":"Recuperados"})

In [ ]:
df

## Conferência dos tipos de variáveis por coluna

In [ ]:
df.dtypes

## Limpeza nos nomes das colunas

In [ ]:
import re   

def corrige_colunas(col_name):  #correcao para eliminar caracteres especiais, maiúsculas, espaços em branco
    return re.sub(r"[/ | ]", "", col_name).lower()  #padrão de escrita das colunas é letra minúscula

In [ ]:
corrige_colunas('AvqO/C  ou')   #teste da correcao

In [ ]:
df.columns = [corrige_colunas(col) for col in df.columns] #lista as colunas, depois atribui novos nomes
df

## Exploração de amostras de dados 

In [ ]:
df.paisregiao.value_counts()   #listagem de 5 primeiros e 5 últimos países

In [ ]:
df.paisregiao.unique()  #listagem de todos os países

In [ ]:
df.loc[df.paisregiao == 'Brazil']  #selecao de amostra do Brasil

In [ ]:
brasil = df.loc[  #seleção com loc é por nome 
    (df.paisregiao == 'Brazil') &
    (df.confirmados > 0)
]

In [ ]:
brasil

## Visualização de casos confirmados

In [ ]:
px.line(brasil, 'datadeobservacao', 'confirmados', title="Casos confirmados no Brasil")
#gera e exibe gráfico de cados confirmados 

## Contagem de novos casos por dia

In [ ]:
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil ['confirmados'].iloc[x] - brasil['confirmados'].iloc[x-1],
    np.arange(brasil.shape[0]) 
))  #pode desconsiderar a advertencia sobre sintaxe, porque a função está criada e nao intere.     

In [ ]:
brasil

## Visualização de novos casos por dia

In [ ]:
px.line(brasil, x='datadeobservacao', y='novoscasos', title="Novos casos por dia")

## Visualização de casos de óbito por dia

In [ ]:
fig = go.Figure() #criacao de figura 

fig.add_trace(
        go.Scatter(x=brasil.datadeobservacao, y=brasil.obitos, name='Óbitos',
                  mode="lines+markers", line={'color':'blue'})
)
#criacao do layout
fig.update_layout(title="Óbitos por COVID-19 no Brasil")

fig.show()

## Cálculo da taxa média de crescimento de casos

In [ ]:
# taxa_crescimento = (presente/passado)**(1/n) - 1
def taxa_crescimento(data, variable, data_inicio = None, data_fim = None):
    #Se data_inicio for none, define com a primeira data disponivel.
    if data_inicio == None:
        data_inicio = data.datadeobservacao.loc[data[variable] > 0].min() #minima data tem ao menos 1 caso
    else:
        data_inicio = pd.to_datetime(data_inicio) #passa a data como string ou texto
    
    if data_fim == None:
        data_fim = data.datadeobservacao.iloc[-1] #passa a ultima data
    else:
        data_fim = pd.to_datetime(data_fim)
        
    #Definição de valores do presente e passado
    passado = data.loc[data.datadeobservacao == data_inicio, variable].values[0]
    presente = data.loc[data.datadeobservacao == data_fim, variable].values[0]
    
    #Definição do numero de pontos no tempo que serão avaliados
    n = (data_fim - data_inicio).days
    
    #Calculo da taxa de crescimento
    taxa = (presente/passado)**(1/n) -1 
    
    return taxa*100 #retorna a taxa em porcentagem       


In [ ]:
#Taxa de crescimento médio do COVID no Brasil em todo o periodo
taxa_crescimento(brasil, 'confirmados')

In [ ]:
df.sample(5)

In [ ]:
#Taxa de crecimento mpedio do COVID no Brasil por dia 
def taxa_crescimento_diaria(data, variable, data_inicio=None):
    #Se data_inicio for none, define com a primeira data disponivel.
    if data_inicio == None:
        data_inicio = data.datadeobservacao.loc[data[variable] > 0].min() #minima data tem ao menos 1 caso
    else:
        data_inicio = pd.to_datetime(data_inicio) #passa a data como string ou texto
        
    data_fim = data.datadeobservacao.max()
    #Define nro de pontos no tempo que serao avaliados
    n = (data_fim - data_inicio).days
    
    #Taxa calculada de variacao de um dia para o outro
    taxas = list(map(
        lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
        range(1, n+1)
    ))
    return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmados')

In [ ]:
tx_dia

In [ ]:
tx_dia

In [ ]:
#Preparação da plotagem do gráfico
primeiro_dia = brasil.datadeobservacao.loc[brasil.confirmados > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.datadeobservacao.max())[1:],
       y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

## Predições

In [ ]:
#Biblioteca para modelagem
from statsmodels.tsa.seasonal import seasonal_decompose #traz tendencia, sazonalidade, ruido
import matplotlib.pyplot as plt

In [ ]:
confirmados = brasil.confirmados
confirmados.index = brasil.datadeobservacao
confirmados

In [ ]:
#Decomposição das series temporais
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12,10))

ax1.plot(res.observed) #serie dos casos observados
ax2.plot(res.trend) #serie das tendencias
ax3.plot(res.seasonal) #serie da sazonalidade
ax4.plot(confirmados.index, res.resid) #serie do ruído
ax4.axhline(0, linestyle= 'dashed', c='black') #traça linha divisoria em zero
plt.show  #exibe graficos

# ARIMA

In [ ]:
# Instalação de biblioteca
pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
modelo = auto_arima(confirmados)


In [ ]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name='Observados'
))
fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), name='Preditos'
))
fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-30', '2020-06-30')  y=modelo.predict(30), name='Forecast'  
))
fig.update_layout(title='Previsão de casos confirmados no Brasil para os proximos 30 dias')
fig.show()

## Modelo de Crescimento

In [ ]:
#instalação da biblioteca fbprophet
!pip install fbprophet 

In [ ]:
from fbprophet import Prophet

In [ ]:
#Preprocessamentos
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

In [ ]:
#renomeando colunas
train.rename(columns={"datadeobservacao":'ds',"confirmados":'y'}, inplace=True)
test.rename(columns={"datadeobservacao":'ds',"confirmados":'y'}, inplace=True)

In [ ]:
#Definir o modelo de crescimento
profeta = Prophet(growth="logistic", changepoints=['2020-03-21','2020-03-30','2020-04-25','2020-05-03','2020-05-10'])

In [ ]:
#Ponto maximo 
pop = 211463256
train['cap'] = pop

In [ ]:
#treina o modelo
profeta.fit(train)

In [ ]:
#Construir previsões para o futuro
future_dates= profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop 
forecast = profeta.predict(future_dates)

In [ ]:
#colocando no grafico
fig = go.Figure

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=test.index, y=test, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()